# TOPIC MODELLING

In [1]:
#importing libraries
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import statistics
import math
import os
import re
import string
import numpy as np
from gensim import models,corpora
import nltk
from nltk.corpus import stopwords
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.metrics import confusion_matrix
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import re
pd.set_option('max_colwidth', 800)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [2]:
#loading dataset
df=pd.read_excel('/content/PMO translated Cleaned data.xlsx')
df.head()

,Survey Answer ID,Q10\nIs there anything else you would like to share with us?,Q11 translated,Q11\nno-translated
0,f91da0c6-602e-40b0-a96e-aef7469e6ebb,Compliments,thanks,thanks
1,3a836d06-fb1c-41a6-99aa-39202beb281c,Comment on the Service,good,good
2,7f6a288d-d2db-42df-9ee2-aba2b0a782ca,Comment on the Service,NICE SERVICES,NICE SERVICESS
3,c3e1f8d4-2956-4734-9571-6c31656484be,Comment on the Service,page response is unstable. rest is all fine,page respsonse is unstable. rest is all fine
4,a3d6ce90-d5e4-4214-8f14-3899d34e8980,Comment on the Service,THANKS,THANKS


In [3]:
df.shape

(29220, 4)

In [4]:
df.columns

Index(['Survey Answer ID',
       'Q10\nIs there anything else you would like to share with us?',
       'Q11 translated', 'Q11\nno-translated'],
      dtype='object')

In [5]:
### dropping unnecessary columns (id and no translated)
df = df.drop(['Survey Answer ID', 'Q10\nIs there anything else you would like to share with us?', 'Q11\nno-translated'], axis=1)
df.head()

,Q11 translated
0,thanks
1,good
2,NICE SERVICES
3,page response is unstable. rest is all fine
4,THANKS


In [6]:
# renaming the column
df.rename(columns = {'Q11 translated': 'text'}, inplace = True)
df.head()

,text
0,thanks
1,good
2,NICE SERVICES
3,page response is unstable. rest is all fine
4,THANKS


### DATA PREPROCESSING AND EDA

In [7]:
# Cleaning Raw tweets
def clean_text(text):
    
    #remove emails
    text = ' '.join([i for i in text.split() if '@' not in i])
    
    #remove web address
    text = re.sub('http[s]?://\S+', '', text)

    #remove brackets and parantheses (and removing the text inside the brackets and parantheses)
    text = re.sub("[\(\[].*?[\)\]]", "", text)

    #remove punctuations
    text = re.sub(r'[^\w\s]', ' ', text)
    
    #Filter to allow only alphabets and numbers
    text = re.sub(r'[^a-zA-Z\']', ' ', text)
    
    #Remove Unicode characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    #remove double spaces 
    text = re.sub('\s+', ' ', text)

    #remove more than 15 characters
    text = re.sub(r'\b\w{16,}\b', '', text)
    
    return text

In [8]:
# applying that into function.
df["clean_text"] = df['text'].apply(lambda x: clean_text(x))
df.head(10)

,text,clean_text
0,thanks,thanks
1,good,good
2,NICE SERVICES,NICE SERVICES
3,page response is unstable. rest is all fine,page response is unstable rest is all fine
4,THANKS,THANKS
5,thanks,thanks
6,System error message,System error message
7,Good,Good
8,page response is unstable. rest is all fine,page response is unstable rest is all fine
9,VERY GOOD AND WE ARE HAPPY KEEP THE GOOD THING GOING GOOD DAY,VERY GOOD AND WE ARE HAPPY KEEP THE GOOD THING GOING GOOD DAY


In [9]:
# dropping the uncleaned text column "text"
df = df.drop(['text'], axis=1)

In [10]:
# renaming the column
df.rename(columns = {'clean_text':'text'}, inplace = True)
df.head()

,text
0,thanks
1,good
2,NICE SERVICES
3,page response is unstable rest is all fine
4,THANKS


In [11]:
import nltk
from nltk.corpus import stopwords
sw_nltk = stopwords.words('english')
print(sw_nltk)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [12]:
# preprocessing the data (removing stopwords, convert the sentence to lower case, and also applying lemmatization to get meaningful words )
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
wnl=WordNetLemmatizer()
def clean_text(review):
  review = review.lower()
  review = review.split()
  review = [wnl.lemmatize(word) for word in review if word not in sw_nltk]
  #review = [wnl.lemmatize(word) for word in review if word not in sw_spacy]
  review = [wnl.lemmatize(word) for word in review if word not in set(stopwords.words('english'))]
  review = ' '.join(review)
  return review

In [13]:
df['text']=df['text'].apply(clean_text)
df.head()

,text
0,thanks
1,good
2,nice service
3,page response unstable rest fine
4,thanks


In [14]:
# removing single characters after cleaning the sentence
df['text'] = df['text'].str.replace(r'\b\w{1,2}\b', '', regex=True)
df.head()

,text
0,thanks
1,good
2,nice service
3,page response unstable rest fine
4,thanks


In [15]:
df = df[df['text'].apply(lambda x: len(x) > 0)]
df.head()

,text
0,thanks
1,good
2,nice service
3,page response unstable rest fine
4,thanks


In [16]:
df.reset_index(inplace=True)

In [17]:
# function to remove names
def remove_names(text):
    # define regular expression pattern to match names
    pattern = re.compile(r'\b(?:abd|abdouli|abdul|abdulaziz|adel|abdullah|abdulrahman|abrar|abeer|abi|adn|adu|ahmad|ahmed|ahood|aht|alabdooli|alabdouli|alachram|alaghbari|alc|alhamdulillah|alhamoor|ali|alia|alif|alkindi|allah|allhumdulliah|allo|alqubaisi|alsalaam|alsoor|alykom|amal|amata|amed|amen|amer|amis|ammar|amoudi|amra|amsat|amwal|ana|andrew|anf|anz|aqsa|atta|atleat|asharj|ashkram|asif|asim|asma|asmaa|awael|awatef|ayash|ayisha|ayman|azhar|azza|badr|badria|badriya|bahia|bakr|balakrishnan|balooshi|balushi|baniyas|bara|baraem|barsh|bastaki|batayneh|bilal|bint|biyyat|brian|bthe|budaiwi|issy|ive|jaddaf|jafar|jafiliya|jahili|jalal|jamal||janan|janoubi|jasmine|jassim|jazak|jazeera|jebel|jeel|jihadjoseline|juma|jumaia|jvc|kalba|kalban|kareem|katyn|kawkaba|keji|kenneth|khabisi|khaili|khaimah|khair|khalaf|khaled|khalid|khalifa|khamis|kharma|khatib|khatm|khattab|khazna|khor|khorfakkan|kjhgfds|kkk|kmz|konain|kumz|kutty|kvito|kyrgyz|dadsad|sddaddad|sdsds||sha|shaer|shafei|shah|shahama|shakhbout|shama|shamikha|shamkha|shamsa|shamsi|shanahan|sharifa|sharq|shatha|shawamekh|shd|shefaa|shehhi|sheikh|sheikha|shiva|shkran|soo|sooo|soooo|sooooo|soooooo|soumya|sadsadsad|sssasss|suhail|suhaila|suhair|sukkah|szhp|taha|tahaluf|taharba|umm|umrah|xia|yabladi|yas|yasmine|yassin|yousef|youssef|youssefni|ypu|yuen|zaabi|zahir|zakher|zaki|zarooni|zakat|zayed|zina|zindabad|ziyad|zubaida|zuhair)\b', re.IGNORECASE)
    return re.sub(pattern, '', text)

# apply the function to the text column
df['text'] = df['text'].apply(remove_names)

In [18]:
# function to remove places
def replace_places(text):
    # define regular expression pattern to match places
    pattern = re.compile(r'\b(?:abu dhabi|abu|dhabi)\b', re.IGNORECASE)
    return re.sub(pattern, 'abudhabi', text)

# apply the function to the text column
df['text'] = df['text'].apply(replace_places)

In [19]:
df.head()

,index,text
0,0,thanks
1,1,good
2,2,nice service
3,3,page response unstable rest fine
4,4,thanks


In [20]:
df['text'][15]

'thanks'

In [21]:
df.shape

(26618, 2)

### Topic detection using BERT

In [22]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 KB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 KB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 92.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 92.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 72.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   

In [23]:
from bertopic import BERTopic
# Dimension reduction
from umap import UMAP

In [24]:
# Initiate UMAP
umap_model = UMAP(n_neighbors=5, 
                  n_components=5, 
                  min_dist=0.0, 
                  metric='cosine', 
                  random_state=100)

In [25]:
# Initiate BERTopic
topic_model = BERTopic(umap_model=umap_model, top_n_words=5, language="english", calculate_probabilities=True, verbose=True)

In [26]:
#model = BERTopic(language="english")
topics, probs = topic_model.fit_transform(df['text'])

Batches:   0%|          | 0/832 [00:00<?, ?it/s]

2023-03-27 09:58:15,762 - BERTopic - Transformed documents to Embeddings
2023-03-27 09:59:20,819 - BERTopic - Reduced dimensionality
2023-03-27 10:09:49,028 - BERTopic - Clustered reduced embeddings


In [27]:
# Get top 10 terms for a topic
topic_model.get_topic(0)

[('thanks', 0.030754519154248074),
 ('welcome', 0.003384839277831409),
 ('sir', 0.0012074000889214429),
 ('thanksall', 0.0011093644476316705),
 ('compliment', 0.0008193018012503474)]

In [28]:
# Visualize top 5 topic keywords
topic_model.visualize_barchart(top_n_topics=5)